In [2]:
# This is a pretty straight forward implementation of character level 
# text generator model. I this imlementation I am going to observe the
# memory feature of Recurrent Neural Network with GRU Cells. 

In [1]:
import tensorflow as tf
import re
import collections
import numpy as np

In [2]:
with open('text_data/science.txt', 'r') as f:
    text= f.read().lower()

# Vocabulary is a collection all the possible character that the text has for examples alphabets, numerics, punctuations etc..

In [3]:
vocab= set(text)

# char2id is a dictionary to map each characters to a unique numeric Id whcih could be fed to the model, and id2char is a dictionary to map a numeric Id to a unique character which is useful to generate characters form the model predictions. 

In [4]:
char2id= dict((c,i) for i,c in enumerate(vocab))
id2char= dict((i,c) for i,c in enumerate(vocab))

In [3]:
# a function to convert list of given characters to Ids.
# function excepts the list of characters and returns the list
# of ids and the character which is not available is the vocabulary
# has a unique id 0.
def id_char(char):
    vec= np.zeros(shape= (len(char),1), dtype= float)
    for i,c in enumerate(char):
        try:
            j= char2id[c]
            vec[i]= j
        except KeyError:
            pass
        
    return vec

# one hot enoding the list of characters
def vectorize_char(char):
    vec= np.zeros(shape= (len(char),len(vocab)), dtype= float)
    for i,c in enumerate(char):
        try:
            j= char2id[c]
            vec[i,j]= 1
        except KeyError:
            pass
        
    return vec

# this function accepts the list of one hot vectors and returns
# the list of crresponding characters. <UKN> is a token for the
# characters not available in the vocabulary
def vector_characterize(vecs):
    chars= ['<UKN>']*vecs.shape[0]
    for i,j in enumerate(np.where(vecs==1)[1]):
        try:
            chars[i]= id2char[j]
        except KeyError:
            pass
        
    return chars

# Modelling the architecture

In [4]:
# this model has 2 hidden layers each with 256 GRU Cells

# The Rnn is truncated at every 100th time step i.e. the network
# has the memory of last 100 time steps and predicts the next character
# on the basis of that.

# the id corresponding to each character is fed to the network
# which generates the one hot encoded vector as an output.

In [6]:
n_hidden= 256
batch_size= 128
time_step= 100
input_dim= 1
output_dim= len(vocab)

In [7]:
X= tf.placeholder(shape= [None,time_step,input_dim], dtype= tf.float32)
Y= tf.placeholder(shape= [None,output_dim], dtype= tf.float32)

In [18]:
w_in= tf.Variable(tf.truncated_normal(shape= [input_dim,n_hidden], dtype= tf.float32, stddev= 0.001))
b_in= tf.Variable(tf.constant(0.001, shape= [n_hidden]))
w_h= tf.Variable(tf.truncated_normal(shape= [n_hidden,n_hidden], dtype= tf.float32, stddev= 0.001))
b_h= tf.Variable(tf.constant(0.001, shape= [n_hidden]))
w_out= tf.Variable(tf.truncated_normal(shape= [n_hidden,output_dim], dtype= tf.float32, stddev= 0.001))
b_out= tf.Variable(tf.constant(0.001, shape= [output_dim]))

In [9]:
X_in= tf.matmul(tf.reshape(X,(-1,input_dim)),w_in)+b_in
X_in= tf.reshape(X_in, (-1,time_step,n_hidden))

In [10]:
cells= tf.contrib.rnn.GRUCell(n_hidden)
multilayer_cell= tf.contrib.rnn.MultiRNNCell([cells,cells], state_is_tuple= True)
init_state= multilayer_cell.zero_state(batch_size, dtype= tf.float32)

In [11]:
output,state= tf.nn.dynamic_rnn(multilayer_cell,X_in, initial_state= init_state)

In [12]:
output= tf.unstack(tf.transpose(output, (1,0,2)))

In [13]:
y_= tf.matmul(output[-1],w_out)+b_out

In [14]:
loss= tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits= y_, labels= Y))
optimize= tf.train.AdamOptimizer(0.001).minimize(loss)

In [19]:
init= tf.global_variables_initializer()
sess= tf.Session()
sess.run(init)

# Training Data Preparation

In [21]:
# convolving straings of length 101 is fed to the list named data
# where first 100 characters will be treated as the input and 
# last charater will be treated as output label.

data= []
for i in range(1000000):
    data.append(text[i:i+101])

# Training the model for 100 iterations

In [42]:
for i in range(100):
    total_loss= 0
    for j in range(5000):
        x_batch= [id_char(data[j][:-1]) for j in 
                  range(batch_size*j,batch_size*(j+1))]
        x_batch= np.array(x_batch)/len(vocab)
        y_batch= [vectorize_char(data[j][-1])[0] for j in 
                  range(batch_size*j,batch_size*(j+1))]
        
        cost, _= sess.run([loss,optimize], feed_dict= {X: x_batch,
                                                       Y: y_batch})
        total_loss+= cost
        print (i,j,cost)
        
    print (i,total_loss/5000)

0 0 1.36516
0 1 1.2794
0 2 1.29269
0 3 1.4068
0 4 1.23392
0 5 1.33589
0 6 1.32947
0 7 1.15825
0 8 1.24022
0 9 1.49351
0 10 1.17856
0 11 1.17314
0 12 1.22453
0 13 1.32807
0 14 1.68694
0 15 1.40974
0 16 1.16296
0 17 1.28363
0 18 1.44976
0 19 1.24673
0 20 1.11806
0 21 0.978162
0 22 1.14001
0 23 1.0376
0 24 1.12817
0 25 1.21925
0 26 1.05165
0 27 1.15203
0 28 1.15099
0 29 0.976296
0 30 1.22036
0 31 1.18896
0 32 1.09047
0 33 1.04147
0 34 1.06374
0 35 1.03092
0 36 1.15202
0 37 1.08152
0 38 1.48396
0 39 1.10641
0 40 1.48074
0 41 1.81371
0 42 1.6866
0 43 1.53843
0 44 1.30168
0 45 1.77655
0 46 1.33477
0 47 1.35238
0 48 1.30472
0 49 1.41891
0 50 1.19156
0 51 1.32111
0 52 1.62624
0 53 1.13464
0 54 0.993445
0 55 0.887731
0 56 1.20294
0 57 1.3541
0 58 1.33152
0 59 1.43174
0 60 1.41093
0 61 1.2342
0 62 1.02083
0 63 1.34408
0 64 1.43617
0 65 1.41178
0 66 1.30895
0 67 0.954312
0 68 1.35491
0 69 1.23705
0 70 1.17311
0 71 1.19922
0 72 0.947524
0 73 1.34395
0 74 1.11612
0 75 1.20659
0 76 1.12464
0 77 1.42

0 597 1.31627
0 598 1.34156
0 599 1.68801
0 600 1.50208
0 601 1.29909
0 602 1.25108
0 603 1.45438
0 604 1.51686
0 605 1.27973
0 606 1.99251
0 607 1.62063
0 608 1.57011
0 609 1.37599
0 610 1.49671
0 611 1.41764
0 612 2.13075
0 613 1.90437
0 614 1.37321
0 615 0.978629
0 616 1.18593
0 617 1.49711
0 618 1.66771
0 619 1.38532
0 620 1.60581
0 621 1.33414
0 622 1.37296
0 623 1.24358
0 624 1.60051
0 625 1.63003
0 626 1.71547
0 627 1.51207
0 628 1.15814
0 629 1.92753
0 630 1.18629
0 631 1.64774
0 632 1.49864
0 633 1.79858
0 634 1.59061
0 635 1.48769
0 636 1.46206
0 637 1.78611
0 638 1.32639
0 639 1.79178
0 640 1.49817
0 641 1.45951
0 642 1.65733
0 643 1.9782
0 644 1.73766
0 645 1.50244
0 646 1.73314
0 647 1.40723
0 648 1.56649
0 649 1.79884
0 650 1.54768
0 651 1.35346
0 652 1.33448
0 653 1.42567
0 654 1.38917
0 655 1.38136
0 656 1.2047
0 657 1.73901
0 658 1.42421
0 659 1.41416
0 660 1.74845
0 661 1.64382
0 662 1.56236
0 663 1.94163
0 664 1.55687
0 665 1.4374
0 666 1.46427
0 667 1.68959
0 668 1.

0 1174 1.57606
0 1175 1.22959
0 1176 1.4019
0 1177 1.22995
0 1178 1.14097
0 1179 1.39181
0 1180 1.53207
0 1181 1.74585
0 1182 1.37777
0 1183 1.4067
0 1184 1.3629
0 1185 1.51377
0 1186 1.68323
0 1187 1.26033
0 1188 1.31949
0 1189 1.27026
0 1190 1.65855
0 1191 1.52769
0 1192 1.10454
0 1193 1.3292
0 1194 1.6473
0 1195 1.32431
0 1196 1.1966
0 1197 1.46618
0 1198 1.46259
0 1199 1.43284
0 1200 1.62597
0 1201 1.07206
0 1202 0.978706
0 1203 1.14238
0 1204 1.24362
0 1205 1.52556
0 1206 1.29275
0 1207 1.22713
0 1208 1.57188
0 1209 1.27666
0 1210 1.9498
0 1211 1.30763
0 1212 1.18372
0 1213 1.43605
0 1214 1.4339
0 1215 1.2913
0 1216 1.31023
0 1217 1.28862
0 1218 1.49869
0 1219 1.42794
0 1220 1.66622
0 1221 1.48017
0 1222 1.48838
0 1223 1.88626
0 1224 1.71217
0 1225 1.82414
0 1226 1.5378
0 1227 1.7503
0 1228 1.20988
0 1229 1.66818
0 1230 1.47509
0 1231 1.8716
0 1232 1.54322
0 1233 1.36998
0 1234 1.62779
0 1235 1.39547
0 1236 1.53075
0 1237 1.46797
0 1238 1.55959
0 1239 1.23822
0 1240 1.34047
0 1241

0 1724 1.79619
0 1725 1.1388
0 1726 1.88579
0 1727 1.39463
0 1728 1.42499
0 1729 1.50145
0 1730 1.39393
0 1731 1.30074
0 1732 1.23615
0 1733 1.49326
0 1734 1.27361
0 1735 1.36013
0 1736 1.3079
0 1737 1.34696
0 1738 1.46713
0 1739 1.34217
0 1740 1.42299
0 1741 1.39875
0 1742 1.61888
0 1743 1.41278
0 1744 1.23983
0 1745 1.65459
0 1746 1.89265
0 1747 1.78675
0 1748 1.63784
0 1749 1.92135
0 1750 1.66139
0 1751 1.50621
0 1752 1.42918
0 1753 1.64706
0 1754 1.20104
0 1755 1.31656
0 1756 1.46712
0 1757 1.17941
0 1758 1.6893
0 1759 1.35227
0 1760 1.43823
0 1761 1.19117
0 1762 1.39501
0 1763 1.46535
0 1764 1.42038
0 1765 1.08132
0 1766 1.43416
0 1767 1.32817
0 1768 1.39093
0 1769 1.25753
0 1770 1.36299
0 1771 1.24333
0 1772 1.41748
0 1773 1.54506
0 1774 1.58772
0 1775 1.07827
0 1776 1.51535
0 1777 1.47776
0 1778 1.29154
0 1779 1.23495
0 1780 1.50366
0 1781 1.66405
0 1782 1.40845
0 1783 1.35898
0 1784 1.546
0 1785 1.3554
0 1786 1.44655
0 1787 1.33212
0 1788 1.19174
0 1789 1.51183
0 1790 1.32633
0

0 2274 1.7562
0 2275 1.57158
0 2276 1.50284
0 2277 1.49588
0 2278 1.29662
0 2279 1.25892
0 2280 1.40385
0 2281 1.19338
0 2282 1.45588
0 2283 1.55195
0 2284 1.21422
0 2285 1.31186
0 2286 1.32912
0 2287 1.9551
0 2288 1.59291
0 2289 1.19931
0 2290 1.56227
0 2291 1.45729
0 2292 1.3754
0 2293 1.37415
0 2294 1.28744
0 2295 1.35174
0 2296 1.12137
0 2297 1.48697
0 2298 1.12561
0 2299 1.45445
0 2300 1.53525
0 2301 1.61562
0 2302 1.1666
0 2303 1.6383
0 2304 1.762
0 2305 1.63175
0 2306 1.40322
0 2307 1.41676
0 2308 1.83527
0 2309 1.92604
0 2310 1.29141
0 2311 1.53442
0 2312 1.71843
0 2313 1.66536
0 2314 1.79839
0 2315 1.36811
0 2316 1.30739
0 2317 1.26906
0 2318 1.13865
0 2319 1.24123
0 2320 1.41014
0 2321 1.58777
0 2322 1.49003
0 2323 1.80797
0 2324 1.52311
0 2325 1.6785
0 2326 1.01779
0 2327 1.39537
0 2328 1.74272
0 2329 1.40671
0 2330 1.38088
0 2331 1.48417
0 2332 1.19689
0 2333 1.43587
0 2334 1.97448
0 2335 1.13079
0 2336 1.47794
0 2337 0.949486
0 2338 1.36836
0 2339 1.48964
0 2340 1.42551
0 

...

8 1395 1.80678
8 1396 1.32735
8 1397 1.28198
8 1398 1.60048
8 1399 1.21163
8 1400 1.27699
8 1401 1.36754
8 1402 1.2234
8 1403 1.63882
8 1404 1.36685
8 1405 1.41534
8 1406 1.2584
8 1407 1.61903
8 1408 1.43986
8 1409 1.41975
8 1410 1.54445
8 1411 1.91241
8 1412 1.5159
8 1413 1.16025
8 1414 1.29059
8 1415 1.41573
8 1416 1.66115
8 1417 1.32977
8 1418 1.27719
8 1419 1.29811
8 1420 1.53173
8 1421 1.371
8 1422 1.42972
8 1423 0.951813
8 1424 1.5324
8 1425 1.57645
8 1426 1.46007
8 1427 1.41894
8 1428 1.41889
8 1429 1.34905
8 1430 1.49094
8 1431 1.48232
8 1432 1.41419
8 1433 1.5132
8 1434 1.60058
8 1435 1.23476
8 1436 1.46784
8 1437 1.87148
8 1438 1.2671
8 1439 1.00703
8 1440 2.41073
8 1441 2.03989
8 1442 1.62564
8 1443 1.36614
8 1444 1.79872
8 1445 1.58244
8 1446 1.6415
8 1447 1.31517
8 1448 1.29025
8 1449 1.68685
8 1450 1.32479
8 1451 1.5345
8 1452 1.39239
8 1453 1.21908
8 1454 1.5744
8 1455 1.29283
8 1456 1.28299
8 1457 1.49384
8 1458 1.28568
8 1459 1.59761
8 1460 1.49487
8 1461 1.6449
8 1462

8 1946 1.32191
8 1947 1.22427
8 1948 1.07819
8 1949 1.15045
8 1950 1.12214
8 1951 1.26309
8 1952 1.34033
8 1953 1.23648
8 1954 1.21826
8 1955 1.14697
8 1956 1.28113
8 1957 1.48087
8 1958 1.10852
8 1959 1.07587
8 1960 1.41197
8 1961 1.35796
8 1962 1.46391
8 1963 1.3953
8 1964 1.29736
8 1965 1.51703
8 1966 1.14203
8 1967 1.4854
8 1968 1.37192
8 1969 1.24206
8 1970 1.30357
8 1971 1.23546
8 1972 1.74595
8 1973 1.18356
8 1974 1.24076
8 1975 1.70405
8 1976 1.24408
8 1977 1.71589
8 1978 1.64052
8 1979 1.56728
8 1980 1.50649
8 1981 1.33845
8 1982 1.28073
8 1983 1.69796
8 1984 1.5134
8 1985 1.53363
8 1986 1.7754
8 1987 1.52731
8 1988 1.4615
8 1989 1.43722
8 1990 1.55282
8 1991 1.75672
8 1992 1.47628
8 1993 1.72095
8 1994 1.39876
8 1995 1.44706
8 1996 1.36029
8 1997 1.58983
8 1998 1.31832
8 1999 1.54821
8 2000 1.35139
8 2001 1.43712
8 2002 1.1413
8 2003 1.36091
8 2004 1.18617
8 2005 1.09353
8 2006 1.40198
8 2007 1.44892
8 2008 1.31127
8 2009 1.72696
8 2010 1.78575
8 2011 1.49078
8 2012 1.58193
8

8 2496 1.36529
8 2497 1.82715
8 2498 1.50349
8 2499 1.55535
8 2500 1.4301
8 2501 1.34372
8 2502 1.34274
8 2503 1.2441
8 2504 1.52541
8 2505 1.49387
8 2506 1.47801
8 2507 1.11162
8 2508 1.1571
8 2509 1.30324
8 2510 1.03358
8 2511 1.25897
8 2512 1.47285
8 2513 1.22693
8 2514 1.59574
8 2515 1.4168
8 2516 1.48807
8 2517 1.55649
8 2518 1.1984
8 2519 1.36321
8 2520 1.58524
8 2521 1.38732
8 2522 1.19228
8 2523 1.23397
8 2524 1.37837
8 2525 1.46947
8 2526 1.46113
8 2527 1.68034
8 2528 1.56916
8 2529 1.29576
8 2530 1.43671
8 2531 1.36107
8 2532 1.48881
8 2533 1.25356
8 2534 1.37626
8 2535 1.53928
8 2536 0.937197
8 2537 1.38329
8 2538 1.55044
8 2539 1.75073
8 2540 1.71625
8 2541 1.34486
8 2542 1.82897
8 2543 1.35166
8 2544 1.69304
8 2545 1.54464
8 2546 1.30898
8 2547 1.37329
8 2548 1.62112
8 2549 1.23966
8 2550 1.61242
8 2551 1.80536
8 2552 1.3669
8 2553 1.6796
8 2554 1.38623
8 2555 1.55805
8 2556 1.60498
8 2557 1.33409
8 2558 1.25545
8 2559 1.18087
8 2560 1.7148
8 2561 1.34861
8 2562 1.39771
8 

8 3046 1.39942
8 3047 1.55003
8 3048 1.10596
8 3049 1.35735
8 3050 1.20113
8 3051 1.20326
8 3052 1.35219
8 3053 1.35729
8 3054 1.2573
8 3055 1.10022
8 3056 1.70066
8 3057 1.34298
8 3058 1.28475
8 3059 1.64818
8 3060 1.28983
8 3061 1.50485
8 3062 1.42586
8 3063 1.28419
8 3064 1.32658
8 3065 1.4034
8 3066 1.50126
8 3067 1.37013
8 3068 1.09907
8 3069 1.01931
8 3070 0.985541
8 3071 1.69935
8 3072 1.54011
8 3073 1.29852
8 3074 1.4844
8 3075 1.27111
8 3076 1.47335
8 3077 1.53531
8 3078 1.6242
8 3079 1.40805
8 3080 1.3104
8 3081 1.45871
8 3082 1.33242
8 3083 1.36536
8 3084 1.19054
8 3085 1.48129
8 3086 1.21137
8 3087 1.45557
8 3088 1.63555
8 3089 1.38309
8 3090 1.17057
8 3091 1.35263
8 3092 1.6868
8 3093 1.63311
8 3094 1.57509
8 3095 1.1529
8 3096 2.19888
8 3097 1.46749
8 3098 1.36912
8 3099 1.54833
8 3100 1.54697
8 3101 1.47297
8 3102 1.39578
8 3103 1.045
8 3104 1.49772
8 3105 1.60662
8 3106 1.25859
8 3107 1.27915
8 3108 0.991042
8 3109 1.20672
8 3110 1.30523
8 3111 1.20305
8 3112 1.41494
8 

KeyboardInterrupt: 

# in_text is a seed text fed to the model on the basis of which the model generates next 500 characters.

In [44]:
in_text= 'on the basis of scientific evidence, this discovery can '
for i in range(500):
    blank= ' '*(time_step-len(in_text))
    in_= blank+in_text
    if len(in_)>100:
        in_= in_[len(in_)-100:]
    x_batch= np.zeros(shape= (batch_size,time_step,input_dim))

    x_batch[0]= id_char(in_)/len(vocab)
    y_pred= sess.run(y_, feed_dict= {X: x_batch.reshape(-1,time_step,input_dim)})
    index= sess.run(tf.argmax(y_pred[0],0))
    char= id2char[index]
    in_text+= char
print (in_text)

on the basis of scientific evidence, this discovery can be an invested the study says says the study in the mass of the same traditions in the same protein the same traditions in the same protein the same traditions in the same protein the same traditions in the same protein the same traditions in the same protein the same traditions in the same protein the same traditions in the same protein the same traditions in the same protein the same traditions in the same protein the same traditions in the same protein the same traditions in the same protein 


In [5]:
# note that the combination of all the characters predicted by the
# model forms actual english words. Although sentences doesn't makes 
# sense but training the network for loger time and optimizing the
# hyperparameters definetly will improve the model.

# Another importent thing that can be noted from the prediction is
# that the theme of the predicted text is rougly scientific which
# is same as the theme of the training data, thus we could use this 
# model to generate text of specific theme and styles. For example
# Shakespere style text can be generated using this model simply by
# trainng the model over the corpus of Shakespere's writtings.